In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Define the path to the input and output files
input_file_path = '/Users/abhayvaghasiya/Desktop/WORK/UMLS_new_train/scores.txt'
output_file_path = '/Users/abhayvaghasiya/Desktop/WORK/UMLS_new_train/scores_modified_2.txt'

# Read the scores from the file
with open(input_file_path, 'r') as file:
    scores = np.array([float(line.strip()) for line in file]).reshape(-1, 1)

# Initialize MinMaxScaler with the desired range
scaler = MinMaxScaler(feature_range=(0.1, 0.9))

# Fit the scaler and transform the scores
scaled_scores = scaler.fit_transform(scores)

# Flatten the array for easier saving
scaled_scores = scaled_scores.flatten()

# Write the scaled scores to the output file
with open(output_file_path, 'w') as file:
    for score in scaled_scores:
        file.write(f"{score}\n")

print(f"Scaled scores have been saved to: {output_file_path}")


In [ ]:
# Import necessary library
import pandas as pd

# Load the data from the text file into a DataFrame
data = pd.read_csv('KGs/UMLS/train.txt', header=None, sep='\t', names=['Head', 'Relation', 'Tail'])

# Prepare a list to store counts
counts = []

# Iterate through each tuple in the DataFrame
for index, row in data.iterrows():
    # Count how many tuples have the same relation and tail
    count = ((data['Relation'] == row['Relation']) & (data['Tail'] == row['Tail'])).sum()
    # Append the count to the list
    counts.append(count)

# Create a new DataFrame from the counts
counts_df = pd.DataFrame(counts, columns=['Count'])

# Save the counts DataFrame to a new text file
counts_df.to_csv('KGs/UMLS/counts.txt', index=False, header=False)

# Print a confirmation message
print("Counts have been saved to counts.txt")
